In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [2]:
# Download stopwords if needed
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Load the dataset
news_df = pd.read_csv('fake_real.csv')  # Make sure the file exists in the directory


In [4]:
news_df.head()

,title,text,subject,date,label
0,John Lewis Has the PERFECT Birthday Message F...,Rep. John Lewis (D-GA) is a civil rights icon....,News,26-Oct-17,0
1,Nazis Whine Over Impending Release Of An Anti...,With the rise of Donald Trump has also come th...,News,08-Oct-17,0
2,Final Republican tax bill slashes U.S. corpora...,WASHINGTON (Reuters) - Congressional Republica...,politicsNews,15-Dec-17,1
3,Obama?s Head Of Disaster Relief TRASHES Trump...,It s no secret that Donald Trump s response to...,News,30-Sep-17,0
4,Nuclear plan backer denies Inauguration Day te...,WASHINGTON (Reuters) - A company promoting a p...,politicsNews,11-Dec-17,1


In [5]:
news_df.tail()

,title,text,subject,date,label
1995,Rupert Murdoch Told Roger Ailes To Help Elect...,A new report has just revealed that Fox News p...,News,19-Jul-17,0
1996,"Republican U.S. Senators Rubio, Lee want boost...",WASHINGTON (Reuters) - Republican U.S. Senator...,politicsNews,09-Nov-17,1
1997,Trump Throws Hissy Fit After New York Times R...,Donald Trump woke up on Wednesday morning and ...,News,28-Jun-17,0
1998,Democrats Give Trump A Big F**ck You After He...,Donald Trump s Twitter feed is always a goldmi...,News,28-Nov-17,0
1999,Factbox: Big-ticket items at center of Congres...,(Reuters) - The U.S. Congress on Thursday appr...,politicsNews,21-Dec-17,1


In [6]:
news_df.shape

(2000, 5)

In [7]:
# Fill missing values with a blank space
news_df = news_df.fillna(' ')


In [8]:
news_df.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [9]:
# Combine the 'title' and 'text' columns into a single content column
news_df['content'] = news_df['title'] + " " + news_df['text']


In [10]:
news_df

,title,text,subject,date,label,content
0,John Lewis Has the PERFECT Birthday Message F...,Rep. John Lewis (D-GA) is a civil rights icon....,News,26-Oct-17,0,John Lewis Has the PERFECT Birthday Message F...
1,Nazis Whine Over Impending Release Of An Anti...,With the rise of Donald Trump has also come th...,News,08-Oct-17,0,Nazis Whine Over Impending Release Of An Anti...
2,Final Republican tax bill slashes U.S. corpora...,WASHINGTON (Reuters) - Congressional Republica...,politicsNews,15-Dec-17,1,Final Republican tax bill slashes U.S. corpora...
3,Obama?s Head Of Disaster Relief TRASHES Trump...,It s no secret that Donald Trump s response to...,News,30-Sep-17,0,Obama?s Head Of Disaster Relief TRASHES Trump...
4,Nuclear plan backer denies Inauguration Day te...,WASHINGTON (Reuters) - A company promoting a p...,politicsNews,11-Dec-17,1,Nuclear plan backer denies Inauguration Day te...
...,...,...,...,...,...,...
1995,Rupert Murdoch Told Roger Ailes To Help Elect...,A new report has just revealed that Fox News p...,News,19-Jul-17,0,Rupert Murdoch Told Roger Ailes To Help Elect...
1996,"Republican U.S. Senators Rubio, Lee want boost...",WASHINGTON (Reuters) - Republican U.S. Senator...,politicsNews,09-Nov-17,1,"Republican U.S. Senators Rubio, Lee want boost..."
1997,Trump Throws Hissy Fit After New York Times R...,Donald Trump woke up on Wednesday morning and ...,News,28-Jun-17,0,Trump Throws Hissy Fit After New York Times R...
1998,Democrats Give Trump A Big F**ck You After He...,Donald Trump s Twitter feed is always a goldmi...,News,28-Nov-17,0,Democrats Give Trump A Big F**ck You After He...


In [11]:
news_df['content']

0        John Lewis Has the PERFECT Birthday Message F...
1        Nazis Whine Over Impending Release Of An Anti...
2       Final Republican tax bill slashes U.S. corpora...
3        Obama?s Head Of Disaster Relief TRASHES Trump...
4       Nuclear plan backer denies Inauguration Day te...
                              ...                        
1995     Rupert Murdoch Told Roger Ailes To Help Elect...
1996    Republican U.S. Senators Rubio, Lee want boost...
1997     Trump Throws Hissy Fit After New York Times R...
1998     Democrats Give Trump A Big F**ck You After He...
1999    Factbox: Big-ticket items at center of Congres...
Name: content, Length: 2000, dtype: object

In [12]:
# Stemming function
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)  # Remove non-alphabetic characters
    stemmed_content = stemmed_content.lower()            # Lowercase the text
    stemmed_content = stemmed_content.split()            # Tokenize by splitting
    stemmed_content = [ps.stem(word) for word in stemmed_content if word not in stopwords.words('english')]  # Stem and remove stopwords
    stemmed_content = ' '.join(stemmed_content)          # Join back into a single string
    return stemmed_content


In [13]:
# Apply stemming to the 'content' column
news_df['content'] = news_df['content'].apply(stemming)


In [8]:
# Separate the features (X) and the target label (y)
X = news_df['content'].values
y = news_df['label'].values


In [9]:
# Convert text data into numerical data using TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)


In [10]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, stratify=y, random_state=1)


In [11]:
# Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [12]:
# Predict on the test set
y_pred = model.predict(X_test_tfidf)


In [13]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 95.25%


In [14]:
INTERFACE


In [13]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Text
from tkinter.font import Font

# Function to classify news
def classify_news():
    user_input = entry.get("1.0", "end-1c")  # Get the user input from the text box
    if not user_input.strip():
        messagebox.showwarning("Input Error", "Please enter some news text.")
        return
    
    user_input_tfidf = vectorizer.transform([user_input])  # Transform input to match training data
    prediction = model.predict(user_input_tfidf)
    
    if prediction == 1:
        result = "This news is Real."
        status_label.config(text="Prediction: Real News", fg="green")
    else:
        result = "This news is Fake."
        status_label.config(text="Prediction: Fake News", fg="red")
    
    messagebox.showinfo("Prediction", result)

# Function to reset the input area and status label
def reset_input():
    entry.delete("1.0", "end")  # Clear the text area
    status_label.config(text="Prediction: Awaiting input...", fg="black")  # Reset status label

# Create the main window
window = tk.Tk()
window.title("Fake News Detector")
window.geometry("600x400")  # Set window size

# Set background color and padding
window.configure(bg="#F0F0F0")
window.resizable(False, False)

# Set font styles
title_font = Font(family="Helvetica", size=16, weight="bold")
label_font = Font(family="Arial", size=12)
button_font = Font(family="Arial", size=10, weight="bold")

# Title Label
title_label = tk.Label(window, text="News Detector", font=title_font, bg="#F0F0F0", pady=10)
title_label.pack()

# Add a label for instructions
label = tk.Label(window, text="Enter the news text below for classification:", font=label_font, bg="#F0F0F0")
label.pack()

# Frame to contain the Text widget and Scrollbar
text_frame = tk.Frame(window, bg="#F0F0F0")
text_frame.pack(pady=10)

# Scrollbar for the text box
scrollbar = Scrollbar(text_frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Text entry box for input (Scrollable)
entry = Text(text_frame, height=10, width=60, font=("Arial", 12), yscrollcommand=scrollbar.set)
entry.pack(side=tk.LEFT)
scrollbar.config(command=entry.yview)

# Classify button with padding and custom font
classify_button = tk.Button(window, text="Classify News", font=button_font, bg="#4CAF50", fg="white", command=classify_news)
classify_button.pack(pady=10)

# Reset button to clear input and status
reset_button = tk.Button(window, text="Reset", font=button_font, bg="#f44336", fg="white", command=reset_input)
reset_button.pack(pady=5)

# Status label to display prediction result dynamically
status_label = tk.Label(window, text="Prediction: Awaiting input...", font=label_font, bg="#F0F0F0", pady=10)
status_label.pack()

# Run the application
window.mainloop()

